In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
#from esda.moran import Moran
from libpysal.weights.contiguity import Queen
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.preprocessing import MinMaxScaler

/home/sara/.local/lib/python3.8/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/tmp/ipykernel_23722/3730080236.py:4: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS directly (

In [2]:
#path1 = r"C:\Users\PcLaptop\Documents\GitHub\Climate-and-conflict\Datasets\som_adm_ocha_itos_20230308_shp\som_admbnda_adm1_ocha_20230308.shp"
path1 = r"/home/sara/Documenti/GitHub/Climate-and-conflict/Datasets/som_adm_ocha_itos_20230308_shp/som_admbnda_adm1_ocha_20230308.shp"
states_gdf = gpd.read_file(path1)

#file1 = r"C:\Users\PcLaptop\Documents\GitHub\Climate-and-conflict\Datasets\ACLED_1997-01-01-2023-07-18_Somalia.csv"
file1 = r"/home/sara/Documenti/GitHub/Climate-and-conflict/Datasets/ACLED_1997-01-01-2023-07-18_Somalia.csv"
dist = pd.read_csv(file1)

#csv1 = r"C:\Users\PcLaptop\Documents\GitHub\Climate-and-conflict\csv\df_lag1_2016_n4_disp.csv"
csv1 = r"/home/sara/Documenti/GitHub/Climate-and-conflict/csv/df_lag1_2016_n4_disp.csv"
csv2 = r"/home/sara/Documenti/GitHub/Climate-and-conflict/csv/df_lags_2016_n4c_disp_d.csv"
df_with_dummies = pd.read_csv(csv2)

asc = r"/home/sara/Documenti/GitHub/Climate-and-conflict/Datasets/gpw_v4_admin_unit_center_points_population_estimates_rev11_som.csv"
#asc = r"C:\Users\PcLaptop\Documents\GitHub\Climate-and-conflict\Datasets\gpw_v4_admin_unit_center_points_population_estimates_rev11_som.csv"
population_density = pd.read_csv(asc)

In [3]:
population_density=population_density.groupby('NAME2')[['UN_2000_DS','UN_2005_DS','UN_2010_DS','UN_2015_DS','UN_2020_DS']].sum()
population_density.rename(index={'Juba Dhexe (Middle)':'Middle_Juba','Juba Hoose (Lower)':'Lower_Juba','Shabelle Dhexe (Middle)':'Middle_Shabelle','Shabelle Hoose (Lower)':'Lower_Shabelle','Woqooyi Galbeed':'Woqooyi_Galbeed'},inplace=True)	

In [7]:
population_density['UN_2001_DS']=population_density['UN_2000_DS']+(population_density['UN_2005_DS']-population_density['UN_2000_DS'])/5
population_density['UN_2002_DS']=population_density['UN_2001_DS']+(population_density['UN_2005_DS']-population_density['UN_2000_DS'])/5
population_density['UN_2003_DS']=population_density['UN_2002_DS']+(population_density['UN_2005_DS']-population_density['UN_2000_DS'])/5
population_density['UN_2004_DS']=population_density['UN_2003_DS']+(population_density['UN_2005_DS']-population_density['UN_2000_DS'])/5
population_density['UN_2006_DS']=population_density['UN_2005_DS']+(population_density['UN_2010_DS']-population_density['UN_2005_DS'])/5
population_density['UN_2007_DS']=population_density['UN_2006_DS']+(population_density['UN_2010_DS']-population_density['UN_2005_DS'])/5
population_density['UN_2008_DS']=population_density['UN_2007_DS']+(population_density['UN_2010_DS']-population_density['UN_2005_DS'])/5
population_density['UN_2009_DS']=population_density['UN_2008_DS']+(population_density['UN_2010_DS']-population_density['UN_2005_DS'])/5
population_density['UN_2011_DS']=population_density['UN_2010_DS']+(population_density['UN_2015_DS']-population_density['UN_2010_DS'])/5
population_density['UN_2012_DS']=population_density['UN_2011_DS']+(population_density['UN_2015_DS']-population_density['UN_2010_DS'])/5
population_density['UN_2013_DS']=population_density['UN_2012_DS']+(population_density['UN_2015_DS']-population_density['UN_2010_DS'])/5
population_density['UN_2014_DS']=population_density['UN_2013_DS']+(population_density['UN_2015_DS']-population_density['UN_2010_DS'])/5
population_density['UN_2016_DS']=population_density['UN_2015_DS']+(population_density['UN_2020_DS']-population_density['UN_2015_DS'])/5
population_density['UN_2017_DS']=population_density['UN_2016_DS']+(population_density['UN_2020_DS']-population_density['UN_2015_DS'])/5
population_density['UN_2018_DS']=population_density['UN_2017_DS']+(population_density['UN_2020_DS']-population_density['UN_2015_DS'])/5
population_density['UN_2019_DS']=population_density['UN_2018_DS']+(population_density['UN_2020_DS']-population_density['UN_2015_DS'])/5
population_density['UN_2021_DS']=population_density['UN_2020_DS']+(population_density['UN_2020_DS']-population_density['UN_2015_DS'])/5
population_density['UN_2022_DS']=population_density['UN_2021_DS']+(population_density['UN_2020_DS']-population_density['UN_2015_DS'])/5

In [8]:
population_density=population_density[['UN_2000_DS','UN_2001_DS','UN_2002_DS','UN_2003_DS','UN_2004_DS','UN_2005_DS','UN_2006_DS','UN_2007_DS','UN_2008_DS','UN_2009_DS','UN_2010_DS','UN_2011_DS','UN_2012_DS','UN_2013_DS','UN_2014_DS','UN_2015_DS','UN_2016_DS','UN_2017_DS','UN_2018_DS','UN_2019_DS','UN_2020_DS','UN_2021_DS','UN_2022_DS']]
population_density.reset_index(inplace=True)
population_density.rename(columns={'NAME2':'admin1','UN_2000_DS':'2000','UN_2001_DS':'2001','UN_2002_DS':'2002','UN_2003_DS':'2003','UN_2004_DS':'2004','UN_2005_DS':'2005','UN_2006_DS':'2006','UN_2007_DS':'2007','UN_2008_DS':'2008','UN_2009_DS':'2009','UN_2010_DS':'2010','UN_2011_DS':'2011','UN_2012_DS':'2012','UN_2013_DS':'2013','UN_2014_DS':'2014','UN_2015_DS':'2015','UN_2016_DS':'2016','UN_2017_DS':'2017','UN_2018_DS':'2018','UN_2019_DS':'2019','UN_2020_DS':'2020','UN_2021_DS':'2021','UN_2022_DS':'2022'},inplace=True)

#pivot the table to have the years as rows
population_density=population_density.melt(id_vars=['admin1'],value_vars=['2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022'],var_name='year',value_name='population_density')
population_density['year']=population_density['year'].astype(int)
#merge the df_with_dummies with the population density for each region for each year
df_with_dummies=pd.merge(df_with_dummies,population_density,how='left',on=['year','admin1'])

/tmp/ipykernel_23722/2035758634.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  population_density.rename(columns={'NAME2':'admin1','UN_2000_DS':'2000','UN_2001_DS':'2001','UN_2002_DS':'2002','UN_2003_DS':'2003','UN_2004_DS':'2004','UN_2005_DS':'2005','UN_2006_DS':'2006','UN_2007_DS':'2007','UN_2008_DS':'2008','UN_2009_DS':'2009','UN_2010_DS':'2010','UN_2011_DS':'2011','UN_2012_DS':'2012','UN_2013_DS':'2013','UN_2014_DS':'2014','UN_2015_DS':'2015','UN_2016_DS':'2016','UN_2017_DS':'2017','UN_2018_DS':'2018','UN_2019_DS':'2019','UN_2020_DS':'2020','UN_2021_DS':'2021','UN_2022_DS':'2022'},inplace=True)


In [9]:
#df_with_dummies = df_with_dummies[df_with_dummies['conflicts']!=0]
df_with_dummies['conflicts_log'] = np.log(df_with_dummies['conflicts']+1)

col1=['TA', 'PA', 'DL', 'TA_lag1',
       'TA_lag2', 'TA_lag3', 'TA_lag4', 'TA_lag5', 'TA_lag6', 'PA_lag1',
       'PA_lag2', 'PA_lag3', 'PA_lag4', 'PA_lag5', 'PA_lag6', 'DL_lag1',
       'DL_lag2', 'DL_lag3', 'DL_lag4', 'DL_lag5', 'DL_lag6', 'conflicts',
       'sum_disp', 'population_density']
#min_max_scaler = MinMaxScaler().fit(dd[col1])
#dd[col1] = min_max_scaler.transform(dd[col1])

min_max_scaler = MinMaxScaler().fit(df_with_dummies[col1])
df_with_dummies[col1] = min_max_scaler.transform(df_with_dummies[col1])

In [30]:
df_with_dummies

,index,time,admin1,tmx,pre,conflicts,year,month,TA,PA,...,Middle_Shabelle_y,Mudug_y,Nugaal_y,Sanaag_y,Sool_y,Togdheer_y,Woqooyi_Galbeed_y,sum_disp,population_density,conflicts_log
0,1380,2016-01,Awdal,30.050001,3.775000,0.041667,2016,1,0.760128,0.237045,...,0,0,6,0,0,0,0,0.000757,0.016448,1.609438
1,1380,2016-01,Bakool,34.450000,1.612500,0.062500,2016,1,0.717408,0.514353,...,0,0,0,0,0,0,0,0.000000,0.011279,1.945910
2,1380,2016-01,Banadir,33.190000,6.130000,0.375000,2016,1,0.504880,0.423418,...,0,0,0,0,0,0,0,0.000000,0.849495,3.610918
3,1380,2016-01,Bari,29.037500,4.045833,0.062500,2016,1,0.985403,0.393283,...,0,0,3,0,0,0,0,0.000264,0.005697,1.945910
4,1380,2016-01,Bay,34.357143,2.657143,0.166667,2016,1,0.575055,0.484029,...,0,0,0,0,0,0,0,0.000970,0.010683,2.833213
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1507,1463,2022-12,Nugaal,29.970005,3.780000,0.010417,2022,12,0.322265,0.373878,...,0,0,1378,0,0,0,0,0.011720,0.000486,0.693147
1508,1463,2022-12,Sanaag,27.188892,1.261111,0.000000,2022,12,0.490590,0.377120,...,0,0,0,465,0,0,0,0.003955,0.000946,0.000000
1509,1463,2022-12,Sool,28.908335,0.441667,0.083333,2022,12,0.445744,0.374292,...,0,170,0,0,514,0,0,0.005817,0.000796,2.197225
1510,1463,2022-12,Togdheer,28.590000,0.770000,0.000000,2022,12,0.581094,0.354426,...,0,142,0,0,83,366,0,0.005026,0.006629,0.000000


In [31]:
df_with_dummies.to_csv(r"/home/sara/Documenti/GitHub/Climate-and-conflict/csv/df_lags_2016_n4c_disp_d_pop_norm.csv",index=False)
#population_density.to_csv(r"/home/sara/Documenti/GitHub/Climate-and-conflict/csv/population_density.csv",index=False)

In [13]:
df_with_dummies['conflicts_pro_capite']=df_with_dummies['conflicts']/df_with_dummies['population_density']

## SPEI

In [14]:
sp = r"/home/sara/Documenti/GitHub/Climate-and-conflict/drought/spei_somalia.csv"
spei = pd.read_csv(sp)

In [15]:
spei['time']=spei['time'].astype(str).str[:7]

In [16]:
#in  column admin1 substitute the name of the regions  'Middle Juba'  with 'Middle_Juba'
spei['admin1']=spei['admin1'].replace('Middle Juba','Middle_Juba')
spei['admin1']=spei['admin1'].replace('Lower Juba','Lower_Juba')
spei['admin1']=spei['admin1'].replace('Middle Shabelle','Middle_Shabelle')
spei['admin1']=spei['admin1'].replace('Lower Shabelle','Lower_Shabelle')
spei['admin1']=spei['admin1'].replace('Woqooyi Galbeed','Woqooyi_Galbeed')


In [17]:
#merge the df_with_dummies with the spei for each region for each year
df_tot=pd.merge(df_with_dummies,spei,how='left',on=['time','admin1'])

In [18]:
# Add Banadir region with tmx and pre as mean of the neighbouring regions

district1 = 'Lower_Shabelle'  
district2 = 'Middle_Shabelle'  

# Calculate the mean tmx and pre for the neighboring districts
sp = df_tot[(df_tot['admin1']==district1) | (df_tot['admin1']==district2)].groupby('time')['spei'].mean()

new_data = pd.DataFrame({ 'admin1': 'Banadir', 'spei': sp}).reset_index()

# Append the new DataFrame to the original DataFrame
df3 = pd.merge(df_tot, new_data, on=['admin1', 'time'], how='outer')

In [19]:
#merge spei_x and spei_y in one column
df3['spei_x'].fillna(df3['spei_y'], inplace=True)
#drop spei_x and spei_y
df3=df3.drop(['spei_y'],axis=1)
df3.rename(columns={'spei_x':'spei'},inplace=True)

In [22]:
df3.to_csv(r'/home/sara/Documenti/GitHub/Climate-and-conflict/csv/df_lags_disp_pop_spei.csv',index=False)